How to test for momentum change in NFL football using a ten year period of data from 2002 to 2012.

First, consolidate data from Advanced Football Analytics Play-By-Play Data by Brian Burke.

In the process of creating this dataframe, I added two columns in Access that would help out with the data crunching.  A) I added a column that simply provided the yards gained IF it was a third down, B) I added another column to immediately trigger "Succes" if there was either more yards gained than needed for the 3rd down OR a touchdown/field goal was made. In the later cases (although there are NOT many field goals attempted on 3rd down) the 3rd down was considered a success in my dataframe.

In [4]:
import pandas as pd
import numpy as np
from scipy import stats
import requests
import csv

%matplotlib inline
import seaborn as sns

In [5]:
NFLDataFrame = pd.read_csv("../data/2002_2012_nfl_pbp_data.csv")
print NFLDataFrame.dtypes
print NFLDataFrame.head(32)
print NFLDataFrame.shape

gameid             object
qtr                 int64
min               float64
sec               float64
off                object
def                object
down              float64
togo              float64
ydline            float64
description        object
3rd Down Yards     object
Successful_3rd     object
offscore          float64
defscore          float64
season            float64
dtype: object
              gameid  qtr   min   sec  off  def  down  togo  ydline  \
0   20120905_DAL@NYG    1  51.0  43.0  NYG  DAL   2.0   8.0    29.0   
1   20120905_DAL@NYG    1  51.0  33.0  DAL  NYG   1.0  10.0    66.0   
2   20120905_DAL@NYG    1  50.0  54.0  DAL  NYG   2.0   8.0    64.0   
3   20120905_DAL@NYG    1  50.0  39.0  DAL  NYG   2.0  13.0    69.0   
4   20120905_DAL@NYG    1  50.0   1.0  DAL  NYG   3.0   4.0    60.0   
5   20120905_DAL@NYG    1  49.0  26.0  DAL  NYG   1.0  10.0    46.0   
6   20120905_DAL@NYG    1  49.0   9.0  DAL  NYG   1.0  15.0    51.0   
7   20120905_DAL@NYG    1  4

C:\Users\mlybeer\AppData\Local\Continuum\anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
NFL_DataFrame = NFLDataFrame.dropna()

Now, I need to add up all the "Successful" 3rd downs, per 'gameid' - per offense - per quarter.

In [7]:
GameCount = NFL_DataFrame.gameid.count()
print GameCount
#This is the amount of records in the DFrame now - without the BLANKS in the core csv file evidently.

389985


In [8]:
datamask = NFL_DataFrame.Successful_3rd == "Success"

ONLY_SUCC = NFL_DataFrame[datamask]
x = ONLY_SUCC.Successful_3rd.unique()
print x 
#ONLY_SUCC.head(15)


['Success']


In [9]:
ByQtr_success = ONLY_SUCC.groupby(["gameid","off","qtr"]).Successful_3rd.count()
print ByQtr_success

gameid            off  qtr
20020905_SF@NYG   NYG  1      1
                       2      4
                       3      1
                       4      4
                  SF   1      1
                       2      2
                       3      1
20020908_ARI@WAS  ARI  1      2
                       2      1
                       3      2
                  WAS  1      3
                       2      2
                       3      1
                       4      2
20020908_ATL@GB   ATL  2      1
                       3      3
                       4      1
                  GB   1      2
                       2      4
                       3      1
20020908_BAL@CAR  BAL  1      1
                       2      1
                       3      2
                       4      2
                  CAR  1      3
                       2      3
                       4      2
20020908_DAL@HOU  DAL  2      1
                       3      1
                  HOU  1      1
             

OK, I will need to incorporate a Pivot Table with the use of an unstack() flavor.  I went to the StackOverflow website and asked the question AFTER creating an account. MLybeer **** BeastMode@24 **** everything is attached to my Gmail account. . . mlybeer1@gmail.com

In [34]:
QUARTER_FRAME = ONLY_SUCC.pivot_table(index=['gameid','off']
                            ,columns=['qtr']
                            ,values=['Successful_3rd']
                            ,fill_value=0
                            ,aggfunc='count').rename(columns={1:'First',2:'Second',3:'Third',4:'Fourth',5:'Overtime'})           
                            
print QUARTER_FRAME                           


                     Successful_3rd                             
qtr                           First Second Third Fourth Overtime
gameid           off                                            
20020905_SF@NYG  NYG              1      4     1      4        0
                 SF               1      2     1      0        0
20020908_ARI@WAS ARI              2      1     2      0        0
                 WAS              3      2     1      2        0
20020908_ATL@GB  ATL              0      1     3      1        0
                 GB               2      4     1      0        0
20020908_BAL@CAR BAL              1      1     2      2        0
                 CAR              3      3     0      2        0
20020908_DAL@HOU DAL              0      1     1      0        0
                 HOU              1      0     1      1        0
20020908_DET@MIA DET              1      2     2      3        0
                 MIA              1      2     1      1        0
20020908_IND@JAC IND     

In [50]:
QUARTER_FRAME["Successful_3rd"].head()

qtr                   First  Second  Third  Fourth  Overtime
gameid           off                                        
20020905_SF@NYG  NYG      1       4      1       4         0
                 SF       1       2      1       0         0
20020908_ARI@WAS ARI      2       1      2       0         0
                 WAS      3       2      1       2         0
20020908_ATL@GB  ATL      0       1      3       1         0

In [65]:
Game_abs_value = QUARTER_FRAME["Successful_3rd"].Fourth + QUARTER_FRAME["Successful_3rd"].Third + QUARTER_FRAME["Successful_3rd"].Second + QUARTER_FRAME["Successful_3rd"].First
Game_total_delta = (QUARTER_FRAME["Successful_3rd"].Fourth + QUARTER_FRAME["Successful_3rd"].Third) - (QUARTER_FRAME["Successful_3rd"].Second + QUARTER_FRAME["Successful_3rd"].First) 
Game_coeff = (Game_total_delta / Game_abs_value) * 0.5
                
#print Game_coeff


In [66]:
First_abs_value = QUARTER_FRAME["Successful_3rd"].Second + QUARTER_FRAME["Successful_3rd"].First
First_total_delta = (QUARTER_FRAME["Successful_3rd"].Second) - (QUARTER_FRAME["Successful_3rd"].First) 
First_coeff = (First_total_delta / First_abs_value) * 0.25
                
#print First_coeff

In [67]:
Second_abs_value = QUARTER_FRAME["Successful_3rd"].Third + QUARTER_FRAME["Successful_3rd"].Fourth
Second_total_delta = (QUARTER_FRAME["Successful_3rd"].Fourth) - (QUARTER_FRAME["Successful_3rd"].Third) 
Second_coeff = (Second_total_delta / Second_abs_value) * 0.25
                
#print Second_coeff

In [82]:
Final_3rd_score = Game_coeff + First_coeff + Second_coeff
print "Maximum Ranking", round(Final_3rd_score.max(),3)
print "Minimum Ranking", round(Final_3rd_score.min(),3)
print "Average Ranking", round(Final_3rd_score.mean(),3)
print "Std. Deviation Ranking", round(Final_3rd_score.std(),3)

Maximum Ranking 0.833
Minimum Ranking -0.875
Average Ranking 0.02
Std. Deviation Ranking 0.297
